In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import  KFold
from sklearn.linear_model import LassoCV,Lasso,MultiTaskLasso
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import scale
import logging

def time_validation(train,time = pd.Timestamp('2014-7-1')):

    return  ((train['time'] - time) < 0).values


def user_validation(train,ratio_val= 0.2):

    index = train['uid'].unique()
    np.random.shuffle(index)
    choosen = index[:len(index)*ratio_val]
    mask = train['uid'].isin(choosen)

    return ~mask.values

def my_score_func(ground_truth,prediction):
    prediction = np.rint(np.expm1(prediction))
    prediction = np.where(prediction<0,0,prediction)
    deviation_f = np.abs(ground_truth[:,0]-prediction[:,0])/(5+ground_truth[:,0]).astype(np.float64)
    deviation_c = np.abs(ground_truth[:,1]-prediction[:,1])/(3+ground_truth[:,1]).astype(np.float64)
    deviation_l  = np.abs(ground_truth[:,2]-prediction[:,2])/(3+ground_truth[:,2]).astype(np.float64)
    precision = sum((ground_truth[:,0]+ground_truth[:,1] + ground_truth[:,2] +1) *  np.where(1-0.5*deviation_f - 0.25*deviation_c - 0.25*deviation_l>0.8,1,0))
    precision = precision / float(sum(ground_truth[:,0]+ground_truth[:,1] + ground_truth[:,2] +1))

    return precision

In [2]:
logging.root.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')

train_x = np.load('processed_data/train_np.npy')
test_x  = np.load('processed_data/test_np.npy')
train_y = np.load('processed_data/target_np.npy')
train_y_class = np.zeros_like(train_y)
train_y_class[train_y[:,0]>1,0] = 1
train_y_class[train_y[:,1]>0,1] = 1
train_y_class[train_y[:,2]>0,2] = 1
train_basic = pd.read_pickle('raw_data/basic_train')
train_basic = train_basic.loc[1626750:]

score = make_scorer(my_score_func,greater_is_better=True)
weight = 1 + train_y[:,0] + train_y[:,1] + train_y[:,2]
train_y = np.log1p(train_y)

In [3]:
lasso = LassoCV(selection='random',n_jobs=2,verbose=2)

In [ ]:
lasso.fit(train_x,train_y[:,0])

In [ ]:
parameters = dict(alpha=[0.003])
regression = GridSearchCV(MultiTaskLasso(selection='random'),parameters,scoring=score,iid=False,cv=5,refit=True,verbose=2)
regression.fit(train_x,train_y)

In [ ]:
parameters = dict(alpha=np.linspace(0.001,0.05,20))
regression = GridSearchCV(MultiTaskEla(selection='random'),parameters,scoring=score,iid=False,cv=5,refit=True,verbose=2)